In [ ]:
from pymongo import MongoClient
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['collaborativefilteringtest']  # Database name
users_collection = db['users']
posts_collection = db['posts']
likes_collection = db['likes']


In [ ]:
# Retrieve data
likes = list(likes_collection.find())

user_ids = list(users_collection.distinct('_id'))
post_ids = list(posts_collection.distinct('_id'))

# Create user-post matrix
user_post_matrix = pd.DataFrame(0, index=user_ids, columns=post_ids)
for like in likes:
    user_post_matrix.at[like['user_id'], like['post_id']] = 1


print('post_id: ', post_ids , 'user_ids: ', user_ids)

# Calculate user similarity
similarity_matrix = cosine_similarity(user_post_matrix)
user_similarities = pd.DataFrame(similarity_matrix, index=user_ids, columns=user_ids)

In [ ]:

# Recommend posts
def recommend_posts(user_id, user_post_matrix, user_similarities, top_n=10):
    similar_users = user_similarities[user_id].sort_values(ascending=False).index[1:top_n+1]
    similar_users_posts = user_post_matrix.loc[similar_users].sum(axis=0)
    recommended_posts = similar_users_posts[similar_users_posts > 0].index.difference(user_post_matrix.loc[user_id][user_post_matrix.loc[user_id] > 0].index)
    return recommended_posts

current_user_id = 'some_user_id'  # replace with the actual user id
recommended_posts = recommend_posts(current_user_id, user_post_matrix, user_similarities)


In [ ]:
# Integrate with search results
def search_posts(query):
    # Placeholder for search logic
    return posts_collection.find({'content': {'$regex': query, '$options': 'i'}})

search_results = search_posts('search_query')
search_results_with_recommendations = list(set(search_results) | set(recommended_posts))